## Treasury data exercise

This notebook shows a quick python script to load data from treasury.gov, then publish a chart pack to PDF.

In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

To get started, we'll retrieve the new data and format it into a pandas DataFrame.

In [2]:
# data source
source = 'https://www.treasury.gov/resource-center/data-chart-center/tic/Documents/slt1d_globl.csv'

# read in raw data
tic = pd.read_csv(source, skiprows=13, skipfooter=10, 
                  names = ('Country Name', 'Country Code', 'End of Month', 
                            'Total US Long-Term Securities', 'US Treasury', 
                            'US Agency Bonds', 'US Corporate and Other Bonds', 
                            'US Corporate Stocks'), 
                  engine='python', thousands=',', na_values = 'n.a.')


# add native date formatting
tic['End of Month'] = pd.to_datetime(tic['End of Month']) + MonthEnd(1)
tic.set_index(['Country Name', 'End of Month'], inplace = True)

tic

Country Code  Total US Long-Term Securities  \
Country Name End of Month                                                
Austria      2018-07-31           10189                          23437   
             2018-06-30           10189                          22761   
             2018-05-31           10189                          22738   
             2018-04-30           10189                          22181   
             2018-03-31           10189                          22787   
             2018-02-28           10189                          22957   
             2018-01-31           10189                          23308   
             2017-12-31           10189                          23185   
             2017-11-30           10189                          22532   
             2017-10-31           10189                          20875   
             2017-09-30           10189                          20767   
             2017-08-31           10189                          20031   
             2017-07-31           10189                          20068   
             2017-06-30           10189                          19844   
             2017-05-31           10189                          19653   
             2017-04-30           10189                          20006   
             2017-03-31           10189                          20319   
             2017-02-28           10189                          19681   
             2017-01-31           10189                          19909   
             2016-12-31           10189                          19736   
             2016-11-30           10189                          19533   
             2016-10-31           10189                          19252   
             2016-09-30           10189                          19508   
             2016-08-31           10189                          19304   
             2016-07-31           10189                          19152   
             2016-06-30           10189                          19299   
             2016-05-31           10189                          18649   
             2016-04-30           10189                          18451   
             2016-03-31           10189                          17626   
             2016-02-29           10189                          16832   
...                                 ...                            ...   
Grand Total  2014-04-30           99996                       15161482   
             2014-03-31           99996                       15110663   
             2014-02-28           99996                       15056278   
             2014-01-31           99996                       14643808   
             2013-12-31           99996                       14723651   
             2013-11-30           99996                       14600714   
             2013-10-31           99996                       14412633   
             2013-09-30           99996                       14138245   
             2013-08-31           99996                       13799980   
             2013-07-31           99996                       13934600   
             2013-06-30           99996                       13655133   
             2013-05-31           99996                       13942791   
             2013-04-30           99996                       13900800   
             2013-03-31           99996                       13731239   
             2013-02-28           99996                       13534206   
             2013-01-31           99996                       13451706   
             2012-12-31           99996                       13216925   
             2012-11-30           99996                       13089818   
             2012-10-31           99996                       12998064   
             2012-09-30           99996                       12985353   
             2012-08-31           99996                       12792089   
             2012-07-31        

Now, let's define a function to draw a table with the most recent data, along with a time series plot for each variable.

In [3]:
def table_plots(country, df):
    """
    country is a string used to title the plots
    df should contain appropriate data
    returns matplotlib.pyplot figure for printing
    """
    fig, ax = plt.subplots(3,2, figsize = (18, 12))
    fig.suptitle(country, fontsize=20)
    for i, x in enumerate(ax.ravel()):

        if i == 0:
            x.table(cellText=list(zip(list(df.iloc[0,1:].index), list(df.iloc[0,1:].values))), 
                    colLabels=['', str(df.index[0])[:-12]],
                    loc='center')
            x.xaxis.set_visible(False) 
            x.yaxis.set_visible(False)
            x.set_title('Most Recent Data (millions USD)')
            continue
        x.grid(True)
        x.plot(df.iloc[:,i].dropna())
        x.set_title(df.iloc[:,i].name)

    return fig


Finally, let's cycle through the countries and draw the tables and plots. 

The output is a PDF with a page for each country.

In [4]:
# list of country names
countries = list(tic.index.levels[0])

# print to pdf
with PdfPages('tic_update.pdf') as pdf:
    for c in countries:

        fig = table_plots(c, tic.loc[c])
        pdf.savefig(fig)

        plt.close()